# Imports

In [1]:
import random
import pandas as pd
import numpy as np

from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, concatenate, Input
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Connect drive

In [3]:
# from google.colab import drive
#
# drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# Dataset

In [2]:
# path_images="/content/drive/My Drive/uc project/images.npz"
path_images='./images.npz'
dict_image_data = np.load(path_images)
image_data = dict_image_data['arr_0']

# path_cities="/content/drive/My Drive/uc project/cities.npz"
path_cities='./cities.npz'
dict_city_data = np.load(path_cities)
city_data = dict_city_data['arr_0']

# path_class="/content/drive/My Drive/uc project/classes.npz"
path_class='./classes.npz'
dict_class_data = np.load(path_class)
class_data = dict_class_data['arr_0']

Set number of data to use from each city and class

In [3]:
n = 10

In [4]:
city_df = pd.DataFrame(city_data, columns = ['city'])
minmax = {}
for city in list(city_df['city'].value_counts().index):
  minmax[city] = {
      'min': city_df[city_df['city'] == city].index[0],
      'max': city_df[city_df['city'] == city].index[-1]
  }
city_indices = np.concatenate([random.sample(range(mm['min'], mm['max']), n) for mm in minmax.values()])

In [5]:
class_df = pd.DataFrame(class_data, columns = ['class'])
minmax = {}
for label in list(class_df['class'].value_counts().index):
  minmax[label] = {
      'min': class_df[class_df['class'] == label].index[0],
      'max': class_df[class_df['class'] == label].index[-1]
  }
class_indices = np.concatenate([random.sample(range(mm['min'], mm['max']), n) for mm in minmax.values()])

Get subset of data

In [6]:
indices = np.concatenate([city_indices, class_indices])
city_data = city_data[indices]
class_data = class_data[indices]

In [7]:
# all 512 images resized to 128
data_512 = image_data[:,0][indices]
# all 512 images cropped to 256 images and resized to 128
data_256 = image_data[:,1][indices]
# all 512 images cropped to 128 images
data_128 = image_data[:,2][indices]

In [8]:
city_data.shape, data_512.shape, data_512.shape

((230,), (230, 128, 128, 3), (230, 128, 128, 3))

Tranforming labels to onehot encoding

In [9]:
from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()

train_y_labels_city = label_as_binary.fit_transform(city_data)
train_y_labels_class = label_as_binary.fit_transform(class_data)

Split train and test data


In [10]:
#City
train_X_512_ci,valid_X_512_ci,train_label_512_ci,valid_label_512_ci = train_test_split(data_512, train_y_labels_city, test_size=0.3)
train_X_256_ci,valid_X_256_ci,train_label_256_ci,valid_label_256_ci = train_test_split(data_256, train_y_labels_city, test_size=0.3)
train_X_128_ci,valid_X_128_ci,train_label_128_ci,valid_label_128_ci = train_test_split(data_128, train_y_labels_city, test_size=0.3)


#Class
train_X_512_cl,valid_X_512_cl,train_label_512_cl,valid_label_512_cl = train_test_split(data_512, train_y_labels_class, test_size=0.3)
train_X_256_cl,valid_X_256_cl,train_label_256_cl,valid_label_256_cl = train_test_split(data_256, train_y_labels_class, test_size=0.3)
train_X_128_cl,valid_X_128_cl,train_label_128_cl,valid_label_128_cl = train_test_split(data_128, train_y_labels_class, test_size=0.3)

In [11]:
x_train_comp = np.stack([data_512,data_256,data_128], axis=1)

x_train_city, x_test_city, y_train_city, y_test_city = train_test_split(x_train_comp, train_y_labels_city, test_size = 0.3)
x_train_class, x_test_claa, y_train_class, y_test_class = train_test_split(x_train_comp, train_y_labels_class, test_size = 0.3)

# Network

In [12]:
def basic_model(input_name, standalone=False):
  model_input = Input(shape=(128, 128, 3), name=input_name)
  conv2d1 = Conv2D(filters=32, kernel_size=(3,3), padding='same',activation='relu')(model_input)
  pool1 = MaxPooling2D((2,2))(conv2d1)
  drop1 = Dropout(0.1)(pool1)
  conv2d2 = Conv2D(filters=32, kernel_size=(3,3),activation='relu')(drop1)
  pool2 = MaxPooling2D((2,2))(conv2d2)
  drop2 = Dropout(0.1)(pool2)
  flat = Flatten()(drop2)
  dense = Dense(1024, activation='relu')(flat)
  if standalone:
    drop = Dropout(0.2)(dense)
    output = Dense(5, activation='softmax')(drop)
    return Model(inputs=[model_input], outputs=[output])
  else:
    output = Dropout(0.2)(dense)
    return model_input, output

## Basic model

In [13]:
model = basic_model('main_input', standalone=True)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(data_512, train_y_labels_city, batch_size=1, epochs=1)

230/230 [==============================] - 25s 102ms/step - loss: 43.9760 - accuracy: 0.2783


## Final model

Three inputs
  1. e128
  1. e256
  1. e512

Two outputs
  1. city
  1. class

Create the three basic input models

In [15]:
e128_input, e128_output = basic_model('e128_input')
e256_input, e256_output = basic_model('e256_input')
e512_input, e512_output = basic_model('e512_input')

Concatenating outputs of three models

In [16]:
merged = concatenate([e128_output, e256_output, e512_output])

Defining the outputs

In [17]:
#City
merged_hidden = Dense(2048, activation='softmax')(merged)
merged_drop = Dropout(0.1)(merged_hidden)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop)

#Class
merged_hidden = Dense(2048, activation='softmax')(merged)
merged_drop = Dropout(0.1)(merged_hidden)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop)

In [18]:
model = Model(inputs=[e128_input, e256_input, e512_input], outputs=[city_output, class_output])
optimizer = keras.optimizers.Adam(learning_rate=0.0004)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Training model

In [19]:
model.fit({'e128_input': data_128, 'e256_input': data_256, 'e512_input': data_512}, {'city_output': train_y_labels_city, 'class_output': train_y_labels_class}, batch_size=1, epochs=10)

Epoch 1/10
230/230 [==============================] - 76s 323ms/step - loss: 4.2870 - city_output_loss: 1.6096 - class_output_loss: 2.6773 - city_output_accuracy: 0.2348 - class_output_accuracy: 0.2478
Epoch 2/10
230/230 [==============================] - 83s 363ms/step - loss: 4.2230 - city_output_loss: 1.6042 - class_output_loss: 2.6188 - city_output_accuracy: 0.2391 - class_output_accuracy: 0.3087
Epoch 3/10
230/230 [==============================] - 65s 283ms/step - loss: 4.1680 - city_output_loss: 1.6025 - class_output_loss: 2.5656 - city_output_accuracy: 0.2304 - class_output_accuracy: 0.3087
Epoch 4/10
230/230 [==============================] - 89s 386ms/step - loss: 4.1170 - city_output_loss: 1.6011 - class_output_loss: 2.5159 - city_output_accuracy: 0.2435 - class_output_accuracy: 0.3087
Epoch 5/10
230/230 [==============================] - 87s 377ms/step - loss: 4.0726 - city_output_loss: 1.5974 - class_output_loss: 2.4752 - city_output_accuracy: 0.2391 - class_output_accurac